In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter

def animar_e_salvar_mp4(caminho_arquivo_entrada):
    """
    Lê os dados da simulação Sol-Terra-Júpiter, gera animação e salva como MP4.
    """

    # ========= 1. Caminhos e arquivos =========
    caminho_absoluto = os.path.abspath(caminho_arquivo_entrada)
    pasta_do_arquivo = os.path.dirname(caminho_absoluto)
    nome_base = os.path.splitext(os.path.basename(caminho_absoluto))[0]
    caminho_saida = os.path.join(pasta_do_arquivo, f"{nome_base}.mp4")

    print(f"Lendo dados de: {caminho_absoluto}")

    # ========= 2. Carregar dados =========
    try:
        data = np.loadtxt(caminho_absoluto, delimiter=',')
    except OSError:
        print(f"Erro: Arquivo não encontrado em {caminho_absoluto}")
        return

    tempo = data[:, 0]
    x_t = data[:, 1]
    y_t = data[:, 2]
    x_j = data[:, 3]
    y_j = data[:, 4]

    # ========= 3. Configurar gráfico =========
    fig, ax = plt.subplots(figsize=(8, 8))

    limite = 6.5  # Júpiter chega em 5.2 UA
    ax.set_xlim(-limite, limite)
    ax.set_ylim(-limite, limite)

    ax.set_aspect('equal')
    ax.grid(True, linestyle='--', alpha=0.5)

    ax.set_xlabel('X (UA)')
    ax.set_ylabel('Y (UA)')
    ax.set_title(f'Simulação: {nome_base}')

    # Sol
    ax.plot(0, 0, 'yo', markersize=12, label='Sol')

    # Terra
    terra, = ax.plot([], [], 'bo', markersize=5, label='Terra')
    rastro_t, = ax.plot([], [], 'b-', linewidth=1, alpha=0.5)

    # Júpiter
    jupiter, = ax.plot([], [], 'ro', markersize=7, label='Júpiter')
    rastro_j, = ax.plot([], [], 'r-', linewidth=1, alpha=0.5)

    texto_tempo = ax.text(0.05, 0.95, '', transform=ax.transAxes)

    ax.legend(loc='upper right')

    # ========= 4. Frames =========
    passo_frame = 10
    frames = range(0, len(tempo), passo_frame)

    def init():
        terra.set_data([], [])
        jupiter.set_data([], [])
        rastro_t.set_data([], [])
        rastro_j.set_data([], [])
        texto_tempo.set_text('')
        return terra, jupiter, rastro_t, rastro_j, texto_tempo

    def update(i):
        # Terra
        terra.set_data([x_t[i]], [y_t[i]])
        rastro_t.set_data(x_t[:i], y_t[:i])

        # Júpiter
        jupiter.set_data([x_j[i]], [y_j[i]])
        rastro_j.set_data(x_j[:i], y_j[:i])

        texto_tempo.set_text(f"Tempo: {tempo[i]:.2f} anos")

        return terra, jupiter, rastro_t, rastro_j, texto_tempo

    anim = FuncAnimation(fig, update, frames=frames, init_func=init, blit=True)

    # ========= 5. Salvar vídeo =========
    print("Gerando vídeo...")
    writer = FFMpegWriter(fps=30, metadata=dict(artist="SimulacaoPython"), bitrate=1800)

    try:
        anim.save(caminho_saida, writer=writer)
        print(f"Vídeo salvo em: {caminho_saida}")
    except Exception as e:
        print(f"Erro ao salvar vídeo: {e}")

    plt.close(fig)

# ----- Execução -----
if __name__ == "__main__":
    animar_e_salvar_mp4("saida.txt")
